*House Pricing: Notebook para Submissão ao Kaggle*

**Gupo:**
* Carlos Gabriel de Freitas - 19.1.4009
* Gabriel Mace dos Santos Ferreira - 19.1.4013
* Filipe Ramos de Souza Santos - 19.1.4027

**Disciplina:** Mineração de Dados (BCC444)

**Professor:** Anderson Almeida Ferreira

In [1]:
import random
import numpy as np
import pandas as pd

from copy import copy
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

## Pré-Processamento

In [2]:
attributes = [
    {
        'name': 'Id',
        'category': 'ordinal_categorical',
        'description': 'Identificação de cada residência no data sheet',
    },
    {
        'name': 'MSSubClass',
        'category': 'nominal_categorical',
        'description': 'Identifica a classificação geral do zoneamento da venda',
    },
    {
        'name': 'MSZoning',
        'category': 'nominal_categorical',
        'description': 'Identifica a classificação geral do zoneamento da venda',
    },
    {
        'name': 'LotFrontage',
        'category': 'continuous_numerical',
        'description': 'Comprimento total das ruas conectadas à propriedade (medido em Linear Feet)',
    },
    {
        'name': 'LotArea',
        'category': 'continuous_numerical',
        'description': 'Área do lote (medido em Square Feet)',
    },
    {
        'name': 'Street',
        'category': 'symmetric_binary',
        'description': 'Tipo de rua de acesso a propriedade',
    },
    {
        'name': 'Alley',
        'category': 'nominal_categorical',
        'description': 'Tipo de beco de acesso à propriedade',
    },
    {
        'name': 'LotShape',
        'category': 'nominal_categorical',
        'description': 'Formato geral da propriedade',
    },
    {
        'name': 'LandContour',
        'category': 'nominal_categorical',
        'description': 'Planicidade da propriedade',
    },
    {
        'name': 'Utilities',
        'category': 'ordinal_categorical',
        'description': 'Tipos de utilidades disponiveis',
    },
    {
        'name': 'LotConfig',
        'category': 'nominal_categorical',
        'description': 'Configuração do lote',
    },
    {
        'name': 'LandSlope',
        'category': 'nominal_categorical',
        'description': 'Inclinação da propriedade',
    },
    {
        'name': 'Neighborhood',
        'category': 'nominal_categorical',
        'description': 'Localizações físicas dentro dos limites da cidade de Ames',
    },
    {
        'name': 'Condition1',
        'category': 'nominal_categorical',
        'description': 'Proximidade da residência à diferentes condições na cidade',
    },
    {
        'name': 'Condition2',
        'category': 'nominal_categorical',
        'description': 'Proximidade da residência à diferentes condições na cidade (caso mais de uma esteja presente)',
    },
    {
        'name': 'BldgType',
        'category': 'nominal_categorical',
        'description': 'Tipo de habitação',
    },
    {
        'name': 'HouseStyle',
        'category': 'nominal_categorical',
        'description': 'Estilo da habitação',
    },
    {
        'name': 'OverallQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia o material total e o acabamento da casa',
    },
    {
        'name': 'OverallCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição geral da casa',
    },
    {
        'name': 'YearBuilt',
        'category': 'discrete_numerical',
        'description': 'Data em que a casa foi construída',
    },
    {
        'name': 'YearRemodAdd',
        'category': 'discrete_numerical',
        'description':'Data em que a casa foi remodelada (idêntico à data de construção se não houverem remodelações ou acréscimos à residência)',
    },
    {
        'name': 'RoofStyle',
        'category': 'nominal_categorical',
        'description': 'Tipo de telhado',
    },
    {
        'name': 'RoofMatl',
        'category': 'nominal_categorical',
        'description': 'Material do telhado',
    },
    {
        'name': 'Exterior1st',
        'category': 'nominal_categorical',
        'description': 'Revestimento exterior em casa',
    },
    {
        'name': 'Exterior2nd',
        'category': 'nominal_categorical',
        'description': 'Revestimento exterior em casa (se há mais de um material)',
    },
    {
        'name': 'MasVnrType',
        'category': 'nominal_categorical',
        'description': 'Tipo de folheado da alvenaria',
    },
    {
        'name': 'MasVnrArea',
        'category': 'continuous_numerical',
        'description': 'Área de folheado da alvenaria (medido em Square Feet)',
    },
    {
        'name': 'ExterQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia a qualidade do material no exterior da casa'
    },
    {
        'name': 'ExterCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição atual do material no exterior da casa',
    },
    {
        'name': 'Foundation',
        'category': 'nominal_categorical',
        'description': 'Tipo de fundação da casa',
    },
    {
        'name': 'BsmtQual',
        'category': 'ordinal_categorical',
        'description': 'Avalia a altura do porão',
    },
    {
        'name': 'BsmtCond',
        'category': 'ordinal_categorical',
        'description': 'Avalia a condição geral do porão',
    },
    {
        'name': 'BsmtExposure',
        'category': 'ordinal_categorical',
        'description': 'Refere-se a walkout ou paredes no nível do jardim',
    },
    {
        'name': 'BsmtFinType1',
        'category': 'ordinal_categorical',
        'description': 'Classificação da área finalizada do porão'
    },
    {
        'name': 'BsmtFinSF1',
        'category': 'discrete_numerical',
        'description': 'Área finalizada Tipo 1 (medida em Square Feet)',
    },
    {
        'name': 'BsmtFinType2',
        'category': 'ordinal_categorical',
        'description': 'Classificação da área finalizada do porão (caso tenham múltiplos tipos)',
    },
    {
        'name': 'BsmtFinSF2',
        'category': 'discrete_numerical',
        'description': 'Área finalizada Tipo 2 (medida em Square Feet)',
    },
    {
        'name': 'BsmtUnfSF',
        'category': 'discrete_numerical',
        'description': 'Área não finalizada do porão (medida em Square Feet)',
    },
    {
        'name': 'TotalBsmtSF',
        'category': 'discrete_numerical',
        'description': 'Área total do porão (medida em Square Feet)',
    },
    {
        'name': 'Heating',
        'category': 'nominal_categorical',
        'description': 'Tipo de Aquecimento',
    },
    {
        'name': 'HeatingQC',
        'category': 'ordinal_categorical',
        'description': 'Qualidade e condição do aquecimento',
    },
    {
        'name': 'CentralAir',
        'category': 'asymmetric_binary',
        'description': 'Ar condicionado central',
    },
    {
        'name': 'Electrical',
        'category': 'nominal_categorical',
        'description': 'Sistema elétrico',
    },
    {
        'name': '1stFlrSF',
        'category': 'discrete_numerical',
        'description': 'Área do primeiro andar (medido em Square Feet)',
    },
    {
        'name': '2ndFlrSF',
        'category': 'discrete_numerical',
        'description': 'Área do segundo andar (medida em Square Feet)',
    },
    {
        'name': 'LowQualFinSF',
        'category': 'discrete_numerical',
        'description': 'Área finalizada em baixa qualidade (medida em Square Feet)',
    },
    {
        'name': 'GrLivArea',
        'category': 'discrete_numerical',
        'description': 'Área habitável acima do nível do solo (medida em Square Feet)',
    },
    {
        'name': 'BsmtFullBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros no porão',
    },
    {
        'name': 'BsmtHalfBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros de hóspede no porão',
    },
    {
        'name': 'FullBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros',
    },
    {
        'name': 'HalfBath',
        'category': 'discrete_numerical',
        'description': 'Número de banheiros de hóspede',
    },
    {
        'name': 'BedroomAbvGr',
        'category': 'discrete_numerical',
        'description': 'Número de quartos acima do nível do solo (Não inclui quartos no porão)',
    },
    {
        'name': 'KitchenAbvGr',
        'category': 'discrete_numerical',
        'description': 'Número de cozinhas acima do nível do solo',
    },
    {
        'name': 'KitchenQual',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da cozinha',
    },
    {
        'name': 'TotRmsAbvGrd',
        'category': 'discrete_numerical',
        'description': 'Número total de cômodos acima do nível do solo (não inclui banheiros)',
    },
    {
        'name': 'Functional',
        'category': 'nominal_categorical',
        'description': 'Funcionalidade da residência (É assumida funcionalidade típica até serem requisitadas investigações)',
    },
    {
        'name': 'Fireplaces',
        'category': 'discrete_numerical',
        'description': 'Número de lareiras na residência',
    },
    {
        'name': 'FireplaceQu',
        'category': 'ordinal_categorical',
        'description': 'Qualidade das lareiras',
    },
    {
        'name': 'GarageType',
        'category': 'nominal_categorical',
        'description': 'Localização da garagem',
    },
    {
        'name': 'GarageYrBlt',
        'category': 'discrete_numerical',
        'description': 'Ano de construção da garagem',
    },
    {
        'name': 'GarageFinish',
        'category': 'ordinal_categorical',
        'description': 'Acabamento interno da garagem',
    },
    {
        'name': 'GarageCars',
        'category': 'discrete_numerical',
        'description': 'Tamanho da garagem (medida em capacidade para armazenar carros)',
    },
    {
        'name': 'GarageArea',
        'category': 'discrete_numerical',
        'description': 'Tamanho da garagem (medida em Square Feet)',
    },
    {
        'name': 'GarageQual',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da garagem',
    },
    {
        'name': 'GarageCond',
        'category': 'ordinal_categorical',
        'description': 'Condição da garagem',
    },
    {
        'name': 'PavedDrive',
        'category': 'nominal_categorical',
        'description': 'Pavimento da entrada da garagem',
    },
    {
        'name': 'WoodDeckSF',
        'category': 'discrete_numerical',
        'description': 'Área do deck de madeira (medida em Square Feet)',
    },
    {
        'name': 'OpenPorchSF',
        'category': 'discrete_numerical',
        'description': 'Área da varanda aberta (medida em Square Feet)',
    },
    {
        'name': 'EnclosedPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda fechada (medida em Square Feet)',
    },
    {
        'name': '3SsnPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda de três estações(medida em Square Feet)',
    },
    {
        'name': 'ScreenPorch',
        'category': 'discrete_numerical',
        'description': 'Área da varanda com telas (medida em Square Feet)',
    },
    {
        'name': 'PoolArea',
        'category': 'discrete_numerical',
        'description': 'Área da piscina (medida em Square Feet)',
    },
    {
        'name': 'PoolQC',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da piscina',
    },
    {
        'name': 'Fence',
        'category': 'ordinal_categorical',
        'description': 'Qualidade da cerca',
    },
    {
        'name': 'MiscFeature',
        'category': 'nominal_categorical',
        'description': 'Recurso variado que não se encaixa em outras categorias',
    },
    {
        'name': 'MiscVal',
        'category': 'continuous_numerical',
        'description': 'Valor monetário do recurso variado (medido em $)',
    },
    {
        'name': 'MoSold',
        'category': 'discrete_numerical',
        'description': 'Mês de venda (MM)',
    },
    {
        'name': 'YrSold',
        'category': 'discrete_numerical',
        'description': 'Ano de venda (YYYY)',
    },
    {
        'name': 'SaleType',
        'category': 'nominal_categorical',
        'description': 'Tipo da venda',
    },
    {
        'name': 'SaleCondition',
        'category': 'nominal_categorical',
        'description': 'Condição para venda',
    },
    {
        'name': 'SalePrice',
        'category': 'continuous_numerical',
        'description': 'Valor da venda (medido em $)',
    },
]

In [3]:
# Compondo listas contendo os nomes dos atributos de cada categoria
nominal_categorical = []
ordinal_categorical = []
discrete_numerical = []
continuous_numerical = []
symmetric_binary = []
asymmetric_binary = []

for attribute in attributes:
    if attribute["category"] == "nominal_categorical":
        nominal_categorical.append(attribute['name'])
    elif attribute["category"] == "ordinal_categorical":
        ordinal_categorical.append(attribute['name'])
    elif attribute["category"] == "discrete_numerical":
        discrete_numerical.append(attribute['name'])
    elif attribute["category"] == "continuous_numerical":
        continuous_numerical.append(attribute['name'])
    elif attribute["category"] == "symmetric_binary":
        symmetric_binary.append(attribute['name'])
    elif attribute["category"] == "asymmetric_binary":
        asymmetric_binary.append(attribute['name'])

numerical = copy(discrete_numerical)
numerical.extend(continuous_numerical)

categorical = copy(nominal_categorical)
categorical.extend(ordinal_categorical)

binary = copy(symmetric_binary)
binary.extend(asymmetric_binary)

In [4]:
def fill_null(df, list, substitute):
    for item in list:
        df[item] = df[item].fillna(substitute)

In [5]:
nominal_na = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 'FireplaceQu','GarageType','GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC','Fence','MiscFeature']
nominal_none = ['MasVnrType']

In [6]:
def lower_bound(df, attribute):
    low = float('inf')
    for index, row in df.iterrows():
        if row[attribute].left < low:
            low = row[attribute].left
    return low

def higher_bound(df, attribute):
    high = 0
    for index, row in df.iterrows():
        if row[attribute].right > high:
            high = row[attribute].right
    return high

def min_max_normalization(df, attribute, attribute_type):
    # Normalizando um atributo cujos valores são números (a coluna como um todo suporta as operações min() e max())
    if attribute_type == 'value':
        df[attribute] = (df[attribute] - df[attribute].min()) / (df[attribute].max() - df[attribute].min())

    # Normalizando um atributo cujos valores são intervalos (cada linha suporta indivudalmente as operações left, mid e right)
    elif attribute_type == 'interval':
        min_value = lower_bound(df, attribute)
        max_value = higher_bound(df, attribute)
        new_column = []

        for index, row in df.iterrows():
            new_column.append((row[attribute].mid - min_value) / (max_value - min_value))
        df[attribute] = new_column

In [7]:
def attribute_to_numerical(df, column):
    new_column = []
    new_values = {}

    counter = 0
    for index, row in df.iterrows():
        old_value = row[column]

        if old_value in new_values:
            new_column.append(new_values[old_value])
        else:
            new_values[old_value] = counter
            new_column.append(counter)
            counter += 1

    df[column] = new_column

In [8]:
# Verificando se os valores nulos de um atributo ocorrem apenas quando um atributo relacionado assume um valor definido
def verification_correlation(df, reason_existence, verified_attribute, value):

    # Selecionando as linhas do dataframe em que o atributo atende a condição desejada, ou seja, assume um valor que indica que
    # o atributo que se deseja verificar é inexistente
    df1 = df.loc[df[reason_existence] == value]

    for attribute in list(df1):

        # Verificando se o attribute analisado atualmente corresponde ao desejado
        if attribute == verified_attribute:

            # A seguir é verificado se o numero de nulos no dataframe filtrado com as condições impostas corresponde ao número de nulos
            # presentes no dataframe normal para aquele atributo
            if df1[attribute].isnull().sum() == df[verified_attribute].isnull().sum():
                return True
            else:
                return False
        else:
            continue

In [9]:
def preprocess_df(df, fill_na, fill_none):

    # Filling in null values
    fill_null(df, fill_na, 'NA')
    fill_null(df, fill_none, 'None')

    # Verifying if null values are correlated with the absence of another attribute's values
    if verification_correlation(df, 'MasVnrType', 'MasVnrArea', 'None'):
        df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
    if verification_correlation(df, 'GarageType', 'GarageYrBlt', 'NA'):
        df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

    # Filling in null values that don't enter the previous categories
    df['LotFrontage'] = df['LotFrontage'].fillna(0)
    df['Electrical'] = df['Electrical'].fillna('SBrkr')

    # Noise removal
    for attribute in continuous_numerical:
        if attribute in df.columns:
            df[attribute] = pd.cut(df[attribute], 100, duplicates='drop')

    # Solving attributes inconsistencies
    house_mean = df.loc[df['HouseStyle'] == '1.5Fin']['2ndFlrSF'].mean()

    df.loc[(df['HouseStyle'] == '1.5Fin') & (df['2ndFlrSF'] == 0)] = df.loc[(df['HouseStyle'] == '1.5Fin') & (df['2ndFlrSF'] == 0)].replace(0, house_mean)

    df.loc[(df['LandContour'] == 'Lvl') & (df['LandSlope'] == 'Sev')] = df.loc[(df['LandContour'] == 'Lvl') & (df['LandSlope'] == 'Sev')].replace('Sev', 'Gtl')

    df.loc[(df['LandContour'] == 'HLS') & (df['LandSlope'] == 'Gtl')] = df.loc[(df['LandContour'] == 'HLS') & (df['LandSlope'] == 'Gtl')].replace('Gtl', 'Sev')

    # Transforming Categorical and Binary attributes to Numerical attributes
    non_numerical = []
    non_numerical.extend(nominal_categorical)
    non_numerical.extend(ordinal_categorical)
    non_numerical.extend(symmetric_binary)
    non_numerical.extend(asymmetric_binary)

    for attribute in non_numerical:
        if attribute in df.columns:
            attribute_to_numerical(df, attribute)

    # Normalizing discrete numerical attributes
    for attribute in discrete_numerical:
        if attribute in df.columns:
            min_max_normalization(df, attribute, 'value')

    # Normalizing continuous numerical attributes
    for attribute in continuous_numerical:
        if attribute in df.columns:
            min_max_normalization(df, attribute, 'interval')

    return df

In [10]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train = train.drop(columns = ['Id'])
train = preprocess_df(train, nominal_na, nominal_none)

In [11]:
X_train, y_train = train.drop('SalePrice', axis=1).values, train['SalePrice'].values
X_train, y_train = np.array(X_train), np.array(y_train)

In [12]:
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
ids = test.pop('Id')
test = preprocess_df(test, nominal_na, nominal_none)

In [13]:
X_test = np.array(test.values)

## Treinando o Modelo

In [14]:
model = XGBRegressor(colsample_bytree=1.0, max_depth=2, n_estimators=250)

In [15]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
train_preds = model.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds)
print(f'MSE no treino: {train_mse}')

MSE no treino: 0.0002639620367423144


In [17]:
preds = model.predict(X_test)

In [18]:
def denormalize_sale_price(normalized_sale_price):
    min_value = 34900
    max_value = 755000
    sale_price = ((max_value - min_value) * normalized_sale_price) + min_value
    return round(sale_price)

In [19]:
# Desnormalizando uma das predições como exemplo
sample = random.choice(preds)
print(f'Normalized Sale Price: {sample}')
print(f'           Sale Price: {denormalize_sale_price(sample)}')

Normalized Sale Price: 0.1980399191379547

           Sale Price: 177509


In [20]:
denormalized_preds = [denormalize_sale_price(x) for x in preds]

In [21]:
output = pd.DataFrame({
    'Id': ids,
    'SalePrice': denormalized_preds,
})

print(output.head())

   Id  SalePrice

0   0     145563

1   1     158537

2   2     239338

3   3     191570

4   4     187149


In [22]:
output.to_csv('output.csv', index=False)